In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn

In [17]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']
df = pd.read_csv(url, names=column_names, na_values='?', comment='\t',
                 sep=' ', skipinitialspace=True)
df.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [18]:
df = df.dropna()
df = df.reset_index(drop=True)

import sklearn
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, train_size=0.8)
train_stats = df_train.describe().transpose()

numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std = train_stats.loc[col_name, 'std']
    df_train_norm[col_name] = (df_train_norm[col_name] - mean) / std
    df_test_norm[col_name] = (df_test_norm[col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
259,18.1,0.351207,0.639242,0.458210,0.524449,-0.208874,78,1
121,24.0,-0.837202,-0.681309,0.192766,-0.356298,-0.620995,73,2
175,23.0,-0.837202,-0.739144,-0.205402,-0.316371,-0.246339,75,2
254,20.5,0.351207,0.321153,-0.072679,0.547935,0.577904,78,1
229,15.5,1.539616,2.007990,2.316324,1.598960,-1.295376,77,1


In [19]:
boundaries = torch.tensor([73, 76, 79])
v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)
v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)
numeric_column_names.append('Model Year Bucketed')

In [45]:
from torch.nn.functional import one_hot
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
origin_encoded = one_hot(torch.tensor(le.fit_transform(df_train_norm['Origin'])))
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.tensor(le.transform(df_test_norm['Origin'])))
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

In [46]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

In [47]:
from torch.utils.data import DataLoader, TensorDataset
train_ds = TensorDataset(x_train, y_train)
batch_size=8
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4)

In [60]:
device = torch.device('cuda')

hidden_units = [8, 4]
input_size = x_train.shape[1]
all_layers = []
for hidden_unit in hidden_units:
    all_layers.append(nn.Linear(input_size, hidden_unit))
    all_layers.append(nn.ReLU())
    input_size = hidden_unit
all_layers.append(nn.Linear(hidden_units[-1], 1))
model = nn.Sequential(*all_layers).to(device)
model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

num_epochs = 200
log_epochs = 10
loss_hist_train = [0] * num_epochs
for epoch in range(num_epochs):
    for x_batch, y_batch in train_dl:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_hist_train[epoch] += loss.item() * y_batch.size(0)
    loss_hist_train[epoch] /= len(train_dl)
    if epoch % log_epochs == 0:
        print('Epoch', epoch, 'Loss', loss_hist_train[epoch])

Epoch 0 Loss 4928.600724792481
Epoch 10 Loss 1972.2146675109864
Epoch 20 Loss 135.667914891243
Epoch 30 Loss 106.8393905505538
Epoch 40 Loss 92.75908324718475
Epoch 50 Loss 83.03881667852401
Epoch 60 Loss 75.17350872159004
Epoch 70 Loss 70.94750447273255
Epoch 80 Loss 68.16595578193665
Epoch 90 Loss 67.04717831611633
Epoch 100 Loss 66.36732441782951
Epoch 110 Loss 67.11019813148305
Epoch 120 Loss 65.26727775377222
Epoch 130 Loss 65.18762569390238
Epoch 140 Loss 65.1852544888854
Epoch 150 Loss 64.5529420375824
Epoch 160 Loss 64.30228745266795
Epoch 170 Loss 63.90436831116676
Epoch 180 Loss 64.12098311856388
Epoch 190 Loss 63.868496894836426


In [65]:
with torch.no_grad():
    pred = model(x_test.to(device).float())[:, 0]
    loss = loss_fn(pred, y_test.to(device))
    print(f'Test MSE: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred, y_test.to(device)).item():.4f}')

Test MSE: 7.4758
Test MAE: 2.1211
